In [14]:
import cv2
import mediapipe as mp
import pyautogui
import time  # Make sure you import the time module

video = cv2.VideoCapture(1)  # Changed camera index to 0

if not video.isOpened():
    print("Error: Camera could not be accessed.")
    exit()

handGesture = mp.solutions.hands.Hands()
drawingTools = mp.solutions.drawing_utils
screenWidth, screenHeight = pyautogui.size()
total_distance = 0
distance_count = 0
prev_time = time.time()  # Initialize prev_time before the loop


while True:
    ret, frame = video.read()
    if not ret:
        print("Failed to grab a frame.")
        break
    frame = cv2.flip(frame, 1)
    frameHeight, frameWidth, _ = frame.shape
    rgbConvertedFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = handGesture.process(rgbConvertedFrame)
    hands = output.multi_hand_landmarks

    if hands:
        for hand in hands:
            drawingTools.draw_landmarks(frame, hand)
            landmarks = hand.landmark
            # Get coordinates for landmarks 8 and 4
            x8, y8 = int(landmarks[8].x * frameWidth), int(landmarks[8].y * frameHeight)
            x4, y4 = int(landmarks[4].x * frameWidth), int(landmarks[4].y * frameHeight)
            # Calculate average position
            avgX, avgY = (x8 + x4) // 2, (y8 + y4) // 2
            # Draw a circle at the average position
            cv2.circle(frame, (avgX, avgY), 15, (255, 0, 255), -1)
            # Convert to screen coordinates
            mousePositionX = screenWidth / frameWidth * avgX
            mousePositionY = screenHeight / frameHeight * avgY
            pyautogui.moveTo(mousePositionX, mousePositionY)

    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    prev_time = curr_time
    cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

    cv2.imshow('Virtual Mouse', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [35]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np

video = cv2.VideoCapture(1)  # Adjust camera index as needed

if not video.isOpened():
    print("Error: Camera could not be accessed.")
    exit()

handGesture = mp.solutions.hands.Hands()
drawingTools = mp.solutions.drawing_utils
screenWidth, screenHeight = pyautogui.size()
update_interval = 0.01  # Time between updates in seconds
last_update_time = 0  # Last time the mouse was updated
sensitivity_multiplier = 2.0  # Increase or adjust for higher sensitivity
prev_frame_time = 0  # For calculating FPS
movement_threshold = 5  # Minimum movement threshold in pixels
prev_avgX, prev_avgY = 0, 0  # Previous average position
smoothing_factor = 0.2  # Smoothing factor for cursor movement
current_mouse_x, current_mouse_y = (0,0)  # Current mouse position

while True:
    ret, frame = video.read()
    if not ret:
        print("Failed to grab a frame.")
        break
    frame = cv2.flip(frame, 1)
    frameHeight, frameWidth, _ = frame.shape
    rgbConvertedFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = handGesture.process(rgbConvertedFrame)
    hands = output.multi_hand_landmarks

    current_time = time.time()

    if hands:
        for hand in hands:
            drawingTools.draw_landmarks(frame, hand)
            landmarks = hand.landmark
            x8, y8 = int(landmarks[8].x * frameWidth), int(landmarks[8].y * frameHeight)
            x4, y4 = int(landmarks[4].x * frameWidth), int(landmarks[4].y * frameHeight)
            avgX, avgY = (x8 + x4) // 2, (y8 + y4) // 2
            cv2.circle(frame, (avgX, avgY), 15, (255, 0, 255), -1)

            movement = np.sqrt((avgX - prev_avgX) ** 2 + (avgY - prev_avgY) ** 2) * sensitivity_multiplier

            if current_time - last_update_time >= update_interval and movement > movement_threshold:
                target_x = (screenWidth / frameWidth * avgX) * sensitivity_multiplier
                target_y = (screenHeight / frameHeight * avgY) * sensitivity_multiplier
                # Smoothened mouse movement
                current_mouse_x += (target_x - current_mouse_x) * smoothing_factor
                current_mouse_y += (target_y - current_mouse_y) * smoothing_factor
                # Adjust mouse position to not exceed screen bounds
                current_mouse_x = min(screenWidth, max(0, current_mouse_x))
                current_mouse_y = min(screenHeight, max(0, current_mouse_y))
                pyautogui.moveTo(current_mouse_x, current_mouse_y)
                last_update_time = current_time
                prev_avgX, prev_avgY = avgX, avgY

    fps = 1 / (current_time - prev_frame_time)
    prev_frame_time = current_time

    # Display FPS on frame
    cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Virtual Mouse', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


In [39]:
import cv2
import mediapipe as mp
import pyautogui
import time

video = cv2.VideoCapture(1)  # Ensure the correct camera index

if not video.isOpened():
    print("Error: Camera could not be accessed.")
    exit()

handGesture = mp.solutions.hands.Hands()
drawingTools = mp.solutions.drawing_utils
screenWidth, screenHeight = pyautogui.size()

prev_time = time.time()

while True:
    ret, frame = video.read()
    if not ret:
        print("Failed to grab a frame.")
        break
    frame = cv2.flip(frame, 1)
    frameHeight, frameWidth, _ = frame.shape
    rgbConvertedFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = handGesture.process(rgbConvertedFrame)
    hands = output.multi_hand_landmarks

    if hands:
        for hand in hands:
            drawingTools.draw_landmarks(frame, hand)
            landmarks = hand.landmark
            for id, landmark in enumerate(landmarks):
                if id == 8:  # Index finger tip
                    x = int(landmark.x * frameWidth)
                    y = int(landmark.y * frameHeight)
                    cv2.circle(frame, (x, y), 30, (0, 255, 255), -1)
                    # Adjust the mouse position mapping to ensure balanced movement
                    mousePositionX = np.interp(x, (0, frameWidth), (0, screenWidth))
                    mousePositionY = np.interp(y, (0, frameHeight), (0, screenHeight))
                    pyautogui.moveTo(mousePositionX, mousePositionY)

    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    prev_time = curr_time
    cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

    cv2.imshow('Virtual Mouse', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
